# Cleaning the SonarQube Issue Data
The goal of this script is to extract the information necessary to use the SonarQube issue tags as a label for prediction.

In [44]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# data import
current_dir = os.getcwd()

# construct path to the project data folder
data_dir = os.path.join(current_dir, '..', '..', 'Data','Sonar_Issues')

# load SonarQube measure data
df1 = pd.read_csv(os.path.join(data_dir, 'SONAR_ISSUES_v1.csv'))
df2 = pd.read_csv(os.path.join(data_dir, 'SONAR_ISSUES_v2.csv'))

In [3]:
df1

,projectID,creationDate,closeDate,creationCommitHash,closeCommitHash,type,squid,component,severity,project,startLine,endLine,resolution,status,message,effort,debt,author
0,commons-daemon,2003-09-04T23:28:19Z,NaN,d3416d3a25b16da3d18b3849522fa96183918e5b,NaN,CODE_SMELL,squid:S00112,org.apache:daemon:src/main/java/org/apache/com...,MAJOR,org.apache:daemon,71.0,71.0,NaN,OPEN,Define and throw a dedicated exception instead...,20,20min,yoavs@apache.org
1,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,265.0,265.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
2,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,259.0,259.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
3,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,261.0,261.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
4,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,278.0,278.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941503,commons-cli,2017-07-29T14:36:30Z,NaN,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,CODE_SMELL,squid:S1192,org.apache:commons-cli:src/test/java/org/apach...,MINOR,org.apache:commons-cli,34.0,34.0,NaN,OPEN,Define a constant instead of duplicating this ...,10,10min,rubin@raaftech.com
1941504,commons-cli,2017-07-29T14:36:30Z,NaN,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,CODE_SMELL,squid:S1192,org.apache:commons-cli:src/test/java/org/apach...,MINOR,org.apache:commons-cli,36.0,36.0,NaN,OPEN,Define a constant instead of duplicating this ...,10,10min,rubin@raaftech.com
1941505,commons-cli,2017-07-29T14:36:30Z,NaN,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,CODE_SMELL,squid:S1192,org.apache:commons-cli:src/test/java/org/apach...,MINOR,org.apache:commons-cli,35.0,35.0,NaN,OPEN,Define a constant instead of duplicating this ...,10,10min,rubin@raaftech.com
1941506,commons-cli,2018-02-26T17:23:40Z,NaN,b0024d482050a08efc36c3cabee37c0af0e57a10,NaN,CODE_SMELL,squid:S00112,org.apache:commons-cli:src/test/java/org/apach...,MAJOR,org.apache:commons-cli,153.0,153.0,NaN,OPEN,Define and throw a dedicated exception instead...,20,20min,deep.alexander@gmail.com


In [4]:
df2

,PROJECT_ID,CREATION_ANALYSIS_KEY,ISSUE_KEY,TYPE,RULE,SEVERITY,STATUS,RESOLUTION,EFFORT,DEBT,...,MESSAGE,COMPONENT,START_LINE,END_LINE,START_OFFSET,END_OFFSET,HASH,FROM_HOTSPOT,NOT_FOUND,CLOSE_ANALYSIS_KEY
0,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcPC4KKKThcBEZ8,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,39.0,39.0,27.0,37.0,NaN,NaN,0,NaN
1,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcRC4KKKThcBEZ9,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,45.0,45.0,27.0,38.0,NaN,NaN,0,NaN
2,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcSC4KKKThcBEZ-,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,51.0,51.0,24.0,37.0,NaN,NaN,0,NaN
3,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcSC4KKKThcBEZ_,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,59.0,59.0,24.0,38.0,NaN,NaN,0,NaN
4,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWd5_rilC4KKKThc-rwE,CODE_SMELL,squid:S1135,INFO,CLOSED,FIXED,NaN,NaN,...,Complete the task associated to this TODO comm...,Apache_Cayenne:framework/cayenne-jdk1.4-unpubl...,33.0,33.0,0.0,71.0,NaN,NaN,0,AWeF1Ou7C4KKKThcBEZc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024609,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV-O48jNFNDElBu,CODE_SMELL,squid:S00117,MINOR,OPEN,NaN,2.0,2.0,...,Rename this local variable to match the regula...,org.apache:zookeeper:src/java/test/org/apache/...,202.0,202.0,18.0,22.0,NaN,NaN,0,NaN
1024610,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV-O48jNFNDElBv,BUG,squid:S1143,BLOCKER,OPEN,NaN,30.0,30.0,...,Remove this return statement from this finally...,org.apache:zookeeper:src/java/test/org/apache/...,157.0,157.0,16.0,22.0,NaN,NaN,0,NaN
1024611,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV_O48jNFNDElBw,CODE_SMELL,squid:S1161,MAJOR,OPEN,NaN,5.0,5.0,...,"Add the ""@Override"" annotation above this meth...",org.apache:zookeeper:src/java/test/org/apache/...,206.0,206.0,24.0,27.0,NaN,NaN,0,NaN
1024612,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV-O48jNFNDElBt,CODE_SMELL,squid:HiddenFieldCheck,MAJOR,OPEN,NaN,5.0,5.0,...,"Rename ""PORT"" which hides the field declared a...",org.apache:zookeeper:src/java/test/org/apache/...,202.0,202.0,18.0,22.0,NaN,NaN,0,NaN


## Filter out only Code Smell Issues

In [5]:
df1 = df1[df1['type'] == 'CODE_SMELL']
df1

,projectID,creationDate,closeDate,creationCommitHash,closeCommitHash,type,squid,component,severity,project,startLine,endLine,resolution,status,message,effort,debt,author
0,commons-daemon,2003-09-04T23:28:19Z,NaN,d3416d3a25b16da3d18b3849522fa96183918e5b,NaN,CODE_SMELL,squid:S00112,org.apache:daemon:src/main/java/org/apache/com...,MAJOR,org.apache:daemon,71.0,71.0,NaN,OPEN,Define and throw a dedicated exception instead...,20,20min,yoavs@apache.org
1,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,265.0,265.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
2,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,259.0,259.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
3,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,261.0,261.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
4,commons-daemon,2003-09-04T23:28:19Z,2010-03-15T08:09:26Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CODE_SMELL,squid:S00122,org.apache:deamon:src/java/org/apache/commons/...,MINOR,org.apache:daemon,278.0,278.0,FIXED,CLOSED,"At most one statement is allowed per line, but...",1,1min,yoavs@apache.org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941503,commons-cli,2017-07-29T14:36:30Z,NaN,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,CODE_SMELL,squid:S1192,org.apache:commons-cli:src/test/java/org/apach...,MINOR,org.apache:commons-cli,34.0,34.0,NaN,OPEN,Define a constant instead of duplicating this ...,10,10min,rubin@raaftech.com
1941504,commons-cli,2017-07-29T14:36:30Z,NaN,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,CODE_SMELL,squid:S1192,org.apache:commons-cli:src/test/java/org/apach...,MINOR,org.apache:commons-cli,36.0,36.0,NaN,OPEN,Define a constant instead of duplicating this ...,10,10min,rubin@raaftech.com
1941505,commons-cli,2017-07-29T14:36:30Z,NaN,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,CODE_SMELL,squid:S1192,org.apache:commons-cli:src/test/java/org/apach...,MINOR,org.apache:commons-cli,35.0,35.0,NaN,OPEN,Define a constant instead of duplicating this ...,10,10min,rubin@raaftech.com
1941506,commons-cli,2018-02-26T17:23:40Z,NaN,b0024d482050a08efc36c3cabee37c0af0e57a10,NaN,CODE_SMELL,squid:S00112,org.apache:commons-cli:src/test/java/org/apach...,MAJOR,org.apache:commons-cli,153.0,153.0,NaN,OPEN,Define and throw a dedicated exception instead...,20,20min,deep.alexander@gmail.com


In [6]:
df2 = df2[df2['TYPE'] == 'CODE_SMELL']
df2

,PROJECT_ID,CREATION_ANALYSIS_KEY,ISSUE_KEY,TYPE,RULE,SEVERITY,STATUS,RESOLUTION,EFFORT,DEBT,...,MESSAGE,COMPONENT,START_LINE,END_LINE,START_OFFSET,END_OFFSET,HASH,FROM_HOTSPOT,NOT_FOUND,CLOSE_ANALYSIS_KEY
0,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcPC4KKKThcBEZ8,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,39.0,39.0,27.0,37.0,NaN,NaN,0,NaN
1,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcRC4KKKThcBEZ9,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,45.0,45.0,27.0,38.0,NaN,NaN,0,NaN
2,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcSC4KKKThcBEZ-,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,51.0,51.0,24.0,37.0,NaN,NaN,0,NaN
3,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWeF1RcSC4KKKThcBEZ_,CODE_SMELL,squid:S1133,INFO,OPEN,NaN,10.0,10.0,...,Do not forget to remove this deprecated code s...,Apache_Cayenne:framework/cayenne-jdk1.5-unpubl...,59.0,59.0,24.0,38.0,NaN,NaN,0,NaN
4,org.apache:cayenne,AWd5_psxC4KKKThc-qK6,AWd5_rilC4KKKThc-rwE,CODE_SMELL,squid:S1135,INFO,CLOSED,FIXED,NaN,NaN,...,Complete the task associated to this TODO comm...,Apache_Cayenne:framework/cayenne-jdk1.4-unpubl...,33.0,33.0,0.0,71.0,NaN,NaN,0,AWeF1Ou7C4KKKThcBEZc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024607,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV6O48jNFNDElBr,CODE_SMELL,common-java:DuplicatedBlocks,MAJOR,OPEN,NaN,30.0,30.0,...,2 duplicated blocks of code must be removed.,org.apache:zookeeper:src/java/test/org/apache/...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1024608,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV-O48jNFNDElBs,CODE_SMELL,squid:EmptyStatementUsageCheck,MINOR,OPEN,NaN,2.0,2.0,...,Remove this empty statement.,org.apache:zookeeper:src/java/test/org/apache/...,218.0,218.0,13.0,14.0,NaN,NaN,0,NaN
1024609,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV-O48jNFNDElBu,CODE_SMELL,squid:S00117,MINOR,OPEN,NaN,2.0,2.0,...,Rename this local variable to match the regula...,org.apache:zookeeper:src/java/test/org/apache/...,202.0,202.0,18.0,22.0,NaN,NaN,0,NaN
1024611,org.apache:zookeeper,AWMdqqQTO48jNFNDElBo,AWMdqrV_O48jNFNDElBw,CODE_SMELL,squid:S1161,MAJOR,OPEN,NaN,5.0,5.0,...,"Add the ""@Override"" annotation above this meth...",org.apache:zookeeper:src/java/test/org/apache/...,206.0,206.0,24.0,27.0,NaN,NaN,0,NaN


## Select only the necessary columns
To save space, columns that do not directly contribute to identifying the projects and commits / analysis or helps with bringing the two database versions together are removed.

In [7]:
df1.columns

Index(['projectID', 'creationDate', 'closeDate', 'creationCommitHash',
       'closeCommitHash', 'type', 'squid', 'component', 'severity', 'project',
       'startLine', 'endLine', 'resolution', 'status', 'message', 'effort',
       'debt', 'author'],
      dtype='object')

In [8]:
df1 = df1[["projectID", "type", "creationDate", "creationCommitHash", "closeCommitHash", "status", "message"]]
df1

,projectID,type,creationDate,creationCommitHash,closeCommitHash,status,message
0,commons-daemon,CODE_SMELL,2003-09-04T23:28:19Z,d3416d3a25b16da3d18b3849522fa96183918e5b,NaN,OPEN,Define and throw a dedicated exception instead...
1,commons-daemon,CODE_SMELL,2003-09-04T23:28:19Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CLOSED,"At most one statement is allowed per line, but..."
2,commons-daemon,CODE_SMELL,2003-09-04T23:28:19Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CLOSED,"At most one statement is allowed per line, but..."
3,commons-daemon,CODE_SMELL,2003-09-04T23:28:19Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CLOSED,"At most one statement is allowed per line, but..."
4,commons-daemon,CODE_SMELL,2003-09-04T23:28:19Z,d3416d3a25b16da3d18b3849522fa96183918e5b,6cbc872eb202dfc27f2eb59b02d953c3deca32c8,CLOSED,"At most one statement is allowed per line, but..."
...,...,...,...,...,...,...,...
1941503,commons-cli,CODE_SMELL,2017-07-29T14:36:30Z,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,OPEN,Define a constant instead of duplicating this ...
1941504,commons-cli,CODE_SMELL,2017-07-29T14:36:30Z,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,OPEN,Define a constant instead of duplicating this ...
1941505,commons-cli,CODE_SMELL,2017-07-29T14:36:30Z,3bc9b84d3ae252800eb234a7b41981a9fef8696d,NaN,OPEN,Define a constant instead of duplicating this ...
1941506,commons-cli,CODE_SMELL,2018-02-26T17:23:40Z,b0024d482050a08efc36c3cabee37c0af0e57a10,NaN,OPEN,Define and throw a dedicated exception instead...


In [9]:
df2.columns

Index(['PROJECT_ID', 'CREATION_ANALYSIS_KEY', 'ISSUE_KEY', 'TYPE', 'RULE',
       'SEVERITY', 'STATUS', 'RESOLUTION', 'EFFORT', 'DEBT', 'TAGS',
       'CREATION_DATE', 'CLOSE_DATE', 'MESSAGE', 'COMPONENT', 'START_LINE',
       'END_LINE', 'START_OFFSET', 'END_OFFSET', 'HASH', 'FROM_HOTSPOT',
       'NOT_FOUND', 'CLOSE_ANALYSIS_KEY'],
      dtype='object')

In [10]:
df2 = df2[["PROJECT_ID", "TYPE", "TAGS", "CREATION_DATE", "CREATION_ANALYSIS_KEY", "STATUS", "MESSAGE"]]
df2

,PROJECT_ID,TYPE,TAGS,CREATION_DATE,CREATION_ANALYSIS_KEY,STATUS,MESSAGE
0,org.apache:cayenne,CODE_SMELL,obsolete,2007-01-21 11:11:02,AWd5_psxC4KKKThc-qK6,OPEN,Do not forget to remove this deprecated code s...
1,org.apache:cayenne,CODE_SMELL,obsolete,2007-01-21 11:11:02,AWd5_psxC4KKKThc-qK6,OPEN,Do not forget to remove this deprecated code s...
2,org.apache:cayenne,CODE_SMELL,obsolete,2007-01-21 11:11:02,AWd5_psxC4KKKThc-qK6,OPEN,Do not forget to remove this deprecated code s...
3,org.apache:cayenne,CODE_SMELL,obsolete,2007-01-21 11:11:02,AWd5_psxC4KKKThc-qK6,OPEN,Do not forget to remove this deprecated code s...
4,org.apache:cayenne,CODE_SMELL,cwe,2007-01-21 11:11:02,AWd5_psxC4KKKThc-qK6,CLOSED,Complete the task associated to this TODO comm...
...,...,...,...,...,...,...,...
1024607,org.apache:zookeeper,CODE_SMELL,pitfall,2016-12-21 22:32:01,AWMdqqQTO48jNFNDElBo,OPEN,2 duplicated blocks of code must be removed.
1024608,org.apache:zookeeper,CODE_SMELL,"cert,misra,unused",2016-12-21 22:32:01,AWMdqqQTO48jNFNDElBo,OPEN,Remove this empty statement.
1024609,org.apache:zookeeper,CODE_SMELL,convention,2016-12-21 22:32:01,AWMdqqQTO48jNFNDElBo,OPEN,Rename this local variable to match the regula...
1024611,org.apache:zookeeper,CODE_SMELL,bad-practice,2016-12-21 22:32:01,AWMdqqQTO48jNFNDElBo,OPEN,"Add the ""@Override"" annotation above this meth..."


## Try to bring together the two database versions
Version 1 doesn't contain any tags in the issue table. Both versions do contain a message though. It is to be investigated whether tags for version 1 can be created from the messages. Assumption: If the tags are consistent with the messages, a certain issue message makes it possible to construct tags for version 1.

## Check consistency of tags with messages for database version 2

In [11]:
check = df2[["TAGS", "MESSAGE"]]
check

,TAGS,MESSAGE
0,obsolete,Do not forget to remove this deprecated code s...
1,obsolete,Do not forget to remove this deprecated code s...
2,obsolete,Do not forget to remove this deprecated code s...
3,obsolete,Do not forget to remove this deprecated code s...
4,cwe,Complete the task associated to this TODO comm...
...,...,...
1024607,pitfall,2 duplicated blocks of code must be removed.
1024608,"cert,misra,unused",Remove this empty statement.
1024609,convention,Rename this local variable to match the regula...
1024611,bad-practice,"Add the ""@Override"" annotation above this meth..."


In [12]:
check['PROCESSED_MESSAGE'] = df2['MESSAGE'].copy()
check

C:\Users\carol\AppData\Local\Temp\ipykernel_15744\3609162278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['PROCESSED_MESSAGE'] = df2['MESSAGE'].copy()


,TAGS,MESSAGE,PROCESSED_MESSAGE
0,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
1,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
2,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
3,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
4,cwe,Complete the task associated to this TODO comm...,Complete the task associated to this TODO comm...
...,...,...,...
1024607,pitfall,2 duplicated blocks of code must be removed.,2 duplicated blocks of code must be removed.
1024608,"cert,misra,unused",Remove this empty statement.,Remove this empty statement.
1024609,convention,Rename this local variable to match the regula...,Rename this local variable to match the regula...
1024611,bad-practice,"Add the ""@Override"" annotation above this meth...","Add the ""@Override"" annotation above this meth..."


In [13]:
# Are messages available for all found code smells? - Yes.
check[check['MESSAGE'].isna()]

,TAGS,MESSAGE,PROCESSED_MESSAGE


### Make the messages comparable
The issue messages contain direct names and numbers to describe variables or lines directly. This leads to messages having a similar goal seeming unique due to the specifics. Names and numbers need to be stripped, so the messages become comparable over the whole column.

In [14]:
def process_string(text: str) -> str:
    """ This function removes variable names that are recognizable by being encased in " " or ' '. """
    if isinstance(text, str):
        # remove substrings between double or single quotes
        processed_text = re.sub(r"(this literal).*", r"\1", text)
        processed_text = re.sub(r'"[^"]*"', '', processed_text)
        processed_text = re.sub(r"'[^']*'", '', processed_text)
        return processed_text.strip()

check['PROCESSED_MESSAGE'] = check['PROCESSED_MESSAGE'].apply(process_string)
check

C:\Users\carol\AppData\Local\Temp\ipykernel_15744\583907064.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['PROCESSED_MESSAGE'] = check['PROCESSED_MESSAGE'].apply(process_string)


,TAGS,MESSAGE,PROCESSED_MESSAGE
0,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
1,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
2,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
3,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
4,cwe,Complete the task associated to this TODO comm...,Complete the task associated to this TODO comm...
...,...,...,...
1024607,pitfall,2 duplicated blocks of code must be removed.,2 duplicated blocks of code must be removed.
1024608,"cert,misra,unused",Remove this empty statement.,Remove this empty statement.
1024609,convention,Rename this local variable to match the regula...,Rename this local variable to match the regula...
1024611,bad-practice,"Add the ""@Override"" annotation above this meth...",Add the annotation above this method signature


In [15]:
# remove numbers from strings
check['PROCESSED_MESSAGE'] = check['PROCESSED_MESSAGE'].str.replace(r'\d+', '', regex=True)
check

C:\Users\carol\AppData\Local\Temp\ipykernel_15744\2890904256.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['PROCESSED_MESSAGE'] = check['PROCESSED_MESSAGE'].str.replace(r'\d+', '', regex=True)


,TAGS,MESSAGE,PROCESSED_MESSAGE
0,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
1,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
2,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
3,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
4,cwe,Complete the task associated to this TODO comm...,Complete the task associated to this TODO comm...
...,...,...,...
1024607,pitfall,2 duplicated blocks of code must be removed.,duplicated blocks of code must be removed.
1024608,"cert,misra,unused",Remove this empty statement.,Remove this empty statement.
1024609,convention,Rename this local variable to match the regula...,Rename this local variable to match the regula...
1024611,bad-practice,"Add the ""@Override"" annotation above this meth...",Add the annotation above this method signature


In [49]:
check["PROCESSED_MESSAGE"].unique().tolist()[:100]

['Do not forget to remove this deprecated code someday.',
 'Complete the task associated to this TODO comment.',
 'Move this constructor to comply with Java Code Conventions.',
 'Reorder the modifiers to comply with the Java Language Specification.',
 'Rename this constant name to match the regular expression .',
 'Complex class',
 'Make this line start at column .',
 'Tab characters found (this is the first occurrence)',
 'Move the array designator from the variable to the type.',
 'Replace all tab characters in this file by sequences of white-spaces.',
 'Make the  logger private static final and rename it to comply with the format .',
 'Rename this class name to match the regular expression .',
 'Define a constant instead of duplicating this literal',
 'Define this declaration on a separate line.',
 'Rename selector  to match the regular expression: ^[a-z][a-z\\-]*$',
 'Remove the declaration of thrown exception  which is a runtime exception.',
 'Rename this local variable to match t

In [17]:
unique_messages = check["PROCESSED_MESSAGE"].unique().tolist()
len(unique_messages)

390

In [18]:
# check whether removing duplicates from the df with only tags and processed_message has the same lengthv
check = check.drop_duplicates(subset=['TAGS', 'PROCESSED_MESSAGE'])
check

,TAGS,MESSAGE,PROCESSED_MESSAGE
0,obsolete,Do not forget to remove this deprecated code s...,Do not forget to remove this deprecated code s...
4,cwe,Complete the task associated to this TODO comm...,Complete the task associated to this TODO comm...
503,convention,Move this constructor to comply with Java Code...,Move this constructor to comply with Java Code...
504,convention,Reorder the modifiers to comply with the Java ...,Reorder the modifiers to comply with the Java ...
506,convention,Rename this constant name to match the regular...,Rename this constant name to match the regular...
...,...,...,...
1018671,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
1019542,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
1021194,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
1022791,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...


In [19]:
# Which processed_messages have different tags?

# group by 'PROCESSED_MESSAGE'
grouped = check.groupby('PROCESSED_MESSAGE')

# filter for groups where the number of unique TAGS sets is greater than 1
messages_with_different_tags = grouped.filter(lambda x: x['TAGS'].nunique() > 1)

messages_with_different_tags

,TAGS,MESSAGE,PROCESSED_MESSAGE
671,convention,Replace all tab characters in this file by seq...,Replace all tab characters in this file by seq...
1012512,"convention,psr2",Replace all tab characters in this file by seq...,Replace all tab characters in this file by seq...


### Comparability: Result
Since the length of a dataframe containing only unique combinations of TAGS and PROCESSED_MESSAGE compared to the length of unique processed messages, it's worth trying to infer tags for known messages for version 1.

## Investigating Tags further
It is important to investigate how many tags there are, how many are missing and what kinds of tags exist in the data.

In [20]:
# How many rows have no tag(s)? - 5.
check[check["TAGS"].isna()]

,TAGS,MESSAGE,PROCESSED_MESSAGE
817,NaN,Missing newline after last element,Missing newline after last element
1206,NaN,Start every element on a separate line.,Start every element on a separate line.
12854,NaN,The Cyclomatic Complexity of this class is 407...,The Cyclomatic Complexity of this class is wh...
25598,NaN,Complete the task associated to this TODO comm...,Complete the task associated to this TODO comm...
91339,NaN,Take the required action to fix the issue indi...,Take the required action to fix the issue indi...


In [21]:
# How many unique tags are there? - 61, inluding combinations.
print(len(check["TAGS"].unique().tolist()))
print(check["TAGS"].unique().tolist())

61
['obsolete', 'cwe', 'convention', 'antipattern,code-smell', 'design', 'format', 'clumsy,redundant,unused', 'clumsy', 'bad-practice', 'clumsy,redundant', nan, 'serialization,unused', 'style', 'bad-practice,misra', 'cert,misra,unused', 'unused', 'bad-practice,obsolete', 'performance', 'pitfall', 'bad-practice,cert', 'cert', 'brain-overload', 'java8', 'misra,pitfall', 'obsolete,pitfall', 'cert,cwe,error-handling', 'html5,obsolete', 'design,performance', 'convention,performance', 'convention,pitfall', 'browser-compatibility,convention', 'cert,pitfall', 'suspicious', 'misra,unused', 'cert,cwe,misra', 'error-handling', 'confusing', 'cert,cwe,misra,suspicious', 'cert,unused', 'cert,clumsy,error-handling', 'cert,misra,pitfall', 'cert,error-handling,suspicious', 'cert,multi-threading', 'cert,cwe,suspicious', 'misra,suspicious', 'bad-practice,unpredictable', 'bad-practice,cert,cwe,error-handling', 'performance,pitfall,sql', 'design,pitfall', 'lock-in,pitfall', 'browser-compatibility', 'perfor

In [22]:
# How many unique tags are there without combinations? - 31.
tag_list = check["TAGS"].unique().tolist()
unique_items = set()

for item in tag_list:
    # check whether item from list is a string
    if isinstance(item, str):
        if ',' in item:
            sub_items = item.split(',')
            for sub_item in sub_items:
                # remove whitespace
                unique_items.add(sub_item.strip())
        else:
            unique_items.add(item.strip())

print(f"The unique items in the list are: {sorted(list(unique_items))}")
print(len(sorted(list(unique_items))))

The unique items in the list are: ['antipattern', 'bad-practice', 'brain-overload', 'browser-compatibility', 'cert', 'clumsy', 'code-smell', 'confusing', 'convention', 'cross-browser', 'cwe', 'design', 'error-handling', 'format', 'html5', 'java8', 'lock-in', 'misra', 'multi-threading', 'obsolete', 'performance', 'pitfall', 'psr2', 'redundant', 'serialization', 'sql', 'style', 'suspicious', 'unpredictable', 'unused', 'user-experience']
31


## Compare messages of the both versions
The goal is to infer the tags for version 1 from the known messages of version 2. It's also important to see how many of the tags have successfully been infered out of all the data there is. For this, the following steps are needed:
* Transform the messages according to version 2, so they can be generalised.
* Compare the transformed messages of version 1 to the known messages from version 2, to see how well they align.
* If there are a lot of common messages, infer the tags.

### Transform the messages of version 2

In [23]:
check2 = pd.DataFrame(df1["message"])
check2["processedMessage"] = df1["message"].copy()
check2

,message,processedMessage
0,Define and throw a dedicated exception instead...,Define and throw a dedicated exception instead...
1,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
2,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
3,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
4,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
...,...,...
1941503,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...
1941504,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...
1941505,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...
1941506,Define and throw a dedicated exception instead...,Define and throw a dedicated exception instead...


In [24]:
# apply transformations
check2['processedMessage'] = check2['processedMessage'].apply(process_string)
check2['processedMessage'] = check2['processedMessage'].str.replace(r'\d+', '', regex=True)
check2

,message,processedMessage
0,Define and throw a dedicated exception instead...,Define and throw a dedicated exception instead...
1,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
2,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
3,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
4,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but..."
...,...,...
1941503,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...
1941504,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...
1941505,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...
1941506,Define and throw a dedicated exception instead...,Define and throw a dedicated exception instead...


### Comparing the messages of version 1 and 2

In [25]:
# How many unique messages are there?
len1 = len(check2["processedMessage"].unique().tolist())
len2 = len(check["PROCESSED_MESSAGE"].unique().tolist())
print(f"Version 1: {len1}")
print(f"Version 2: {len2}")

Version 1: 340
Version 2: 390


In [26]:
set1 = set(list(check["PROCESSED_MESSAGE"]))
set2 = set(list(check2["processedMessage"]))

common_elements = list(set1.intersection(set2))
common_elements

['T is not used in the method.',
 'Refactor this method to throw at most one checked exception instead of: java.security.InvalidKeyException, javax.xml.crypto.dsig.XMLSignatureException',
 'Immediately return this expression instead of assigning it to the temporary variable .',
 'Use  for this string-to-short conversion.',
 'Refactor this method to throw at most one checked exception instead of: java.io.IOException, java.io.FileNotFoundException, org.xml.sax.SAXException, javax.xml.parsers.ParserConfigurationException, javax.xml.transform.TransformerException',
 'Use  for this string-to-boolean conversion.',
 'Remove the declaration of thrown exception  which is a subclass of .',
 'Rename  which hides the field declared at line .',
 'This file  should be located in  directory, not in .',
 'Call the static method Double.toString(...) instead of instantiating a temporary object to perform this to string conversion.',
 'is a .',
 'Refactor this method to throw at most one checked exceptio

In [27]:
len(common_elements)

268

Version 1 has 482 different issue messages and version 2 has 517. They only share 339, so for these messages only there can be tags infered from version 2. <br>
Which messages of version 1 are not shared by version 2?

In [28]:
exclusive_in_set2 = list(set2.difference(set1))
exclusive_in_set2

['Refactor this method to throw at most one checked exception instead of: org.apache.aurora.common.zookeeper.Group$JoinException, java.lang.InterruptedException',
 'Refactor this method to throw at most one checked exception instead of: org.apache.aurora.common.zookeeper.ZooKeeperClient$ZooKeeperConnectionException, java.lang.InterruptedException, java.util.concurrent.TimeoutException',
 'Refactor this method to throw at most one checked exception instead of: java.io.IOException, javax.xml.bind.JAXBException',
 'BizzleT is not used in the class.',
 'ViewT is not used in the interface.',
 'Move the true\\ string literal on the left side of this string comparison.',
 'InputT is not used in the class.',
 'Refactor this method to throw at most one checked exception instead of: java.security.cert.CertificateException, java.io.UnsupportedEncodingException',
 'Remove the parentheses around the  parameter (sonar.java.source not set. Assuming  or greater.)',
 'Refactor this method to throw at m

A lot of these messages contain similar parts. So they are checked one-by-one so their tags might be correctly infered even though they aren't in version 2 directly.

In [29]:
check[check['PROCESSED_MESSAGE'].str.contains("is not used in the method.")]

,TAGS,MESSAGE,PROCESSED_MESSAGE
325030,"cert,unused",R is not used in the method.,R is not used in the method.
325500,"cert,unused",T is not used in the method.,T is not used in the method.
1005536,"cert,unused",N is not used in the method.,N is not used in the method.


In [30]:
pd.set_option('display.max_rows', None)
check[check['PROCESSED_MESSAGE'].str.contains("Refactor this method to throw at most one checked exception instead of:")]

,TAGS,MESSAGE,PROCESSED_MESSAGE
9754,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
9950,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
11828,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
15573,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
23042,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
23043,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
23287,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
23768,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
98179,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...
102933,error-handling,Refactor this method to throw at most one chec...,Refactor this method to throw at most one chec...


In [31]:
check[check['PROCESSED_MESSAGE'].str.contains("is not used in the class.")]

,TAGS,MESSAGE,PROCESSED_MESSAGE
320955,"cert,unused",C is not used in the class.,C is not used in the class.
398909,"cert,unused",K is not used in the class.,K is not used in the class.
398910,"cert,unused",V is not used in the class.,V is not used in the class.
456948,"cert,unused",I is not used in the class.,I is not used in the class.
662034,"cert,unused",T is not used in the class.,T is not used in the class.
1003813,"cert,unused",E is not used in the class.,E is not used in the class.


In [32]:
check[check['PROCESSED_MESSAGE'].str.contains("Remove useless curly braces around statement")]

,TAGS,MESSAGE,PROCESSED_MESSAGE


In [33]:
check[check['PROCESSED_MESSAGE'].str.contains("literal on the left side of this string comparison.")]

,TAGS,MESSAGE,PROCESSED_MESSAGE
10063,bad-practice,"Move the ""NSNumber"" string literal on the left...",Move the string literal on the left side of t...
327427,bad-practice,"Move the ""\"""" string literal on the left side ...","Move the "" string literal on the left side of ..."
622917,bad-practice,"Move the ""\""java.lang.Enum\"""" string literal o...",Move the java.lang.Enum\ string literal on the...


In [34]:
check[check['PROCESSED_MESSAGE'].str.contains("Remove the parentheses around the  parameter")]

,TAGS,MESSAGE,PROCESSED_MESSAGE


In [35]:
check[check['PROCESSED_MESSAGE'].str.contains("Reduce the number of non-empty switch cases")]

,TAGS,MESSAGE,PROCESSED_MESSAGE


In [36]:
check[check['PROCESSED_MESSAGE'].str.contains("is not used in the interface.")]

,TAGS,MESSAGE,PROCESSED_MESSAGE
381112,"cert,unused",T is not used in the interface.,T is not used in the interface.
397881,"cert,unused",K is not used in the interface.,K is not used in the interface.
397882,"cert,unused",V is not used in the interface.,V is not used in the interface.
835843,"cert,unused",BatchKey is not used in the interface.,BatchKey is not used in the interface.


In [37]:
check[check['PROCESSED_MESSAGE'].str.contains("method to this class.")]

,TAGS,MESSAGE,PROCESSED_MESSAGE


For all messages only contained in the version 1 data, there is either a consistent tag that it is associated with or no tag that can be infered.

### Check percentage coverage of messages from version 1 that can be infered

In [38]:
pd.reset_option('display.max_rows')

# create a variable to check whether tags can be inferred from version 2
def can_be_inferred(row):
    message = row['processedMessage']
    if message in common_elements:
        return True
    if "is not used in the method." in message:
        return True
    if "Refactor this method to throw at most one checked exception instead of:" in message:
        return True
    if "is not used in the class." in message:
        return True
    if "literal on the left side of this string comparison." in message:
        return True
    if "is not used in the interface." in message:
        return True
    return False


check2['canBeInferred'] = check2.apply(can_be_inferred, axis=1)
check2

,message,processedMessage,canBeInferred
0,Define and throw a dedicated exception instead...,Define and throw a dedicated exception instead...,True
1,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but...",True
2,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but...",True
3,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but...",True
4,"At most one statement is allowed per line, but...","At most one statement is allowed per line, but...",True
...,...,...,...
1941503,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...,True
1941504,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...,True
1941505,Define a constant instead of duplicating this ...,Define a constant instead of duplicating this ...,True
1941506,Define and throw a dedicated exception instead...,Define and throw a dedicated exception instead...,True


In [39]:
# percentage of True values
print(f"Percentages of messages for which tags can be inferred: {round((check2['canBeInferred'].sum() / len(check2)) * 100, 2)} %")

Percentages of messages for which tags can be inferred: 99.98 %


### Result
Following the investigation, it can be concluded that almost all tags can be inferred from version 2, which increases the training data significantly.

In [40]:
# save check dfs
check.to_csv(os.path.join(data_dir, 'issues_v2.csv'), index = False)
check2.to_csv(os.path.join(data_dir, 'issues_v1.csv'), index = False)

## Prepare Version 2 data for tag prediction

### Flattening the labels over different analysis / commits
When pushing a commit and running the analysis, there can be multiple issues for one commit. For each of these commits, there can be multiple code smells. Each of these code smells can have multiple tags. There can be multiple code smells of the same type, so that in theory there would be a distribution of tags in comparison to all tags that SonarQube can assign to an issue. To make this predictable, the approach is to collect all tags for code smells for one commit and make this list the label. For this, the distribution of open (not fixed) code smells need to be flatted into a list.

In [41]:
# select rows for which the issue is not fixed yet
df2 = df2[df2["STATUS"] == "OPEN"]

# select columns that are needed to extract unique tags per analysis
df_tags = df2.groupby('CREATION_ANALYSIS_KEY')['TAGS'].unique().reset_index(name='UNIQUE_TAGS')
df_tags

,CREATION_ANALYSIS_KEY,UNIQUE_TAGS
0,AV0-6-3jt6tne_r58pUF,"[bad-practice,cert, cert,error-handling,suspic..."
1,AV0-8Tovt6tne_r58pUY,[clumsy]
2,AV0-9p72t6tne_r58pUo,[bad-practice]
3,AV0-BQ2Et6tne_r58pCR,[design]
4,AV0-LoN2t6tne_r58pFZ,"[cert,pitfall, cert]"
...,...,...
9688,AWentw92Z1vVHhGU4u0K,"[bad-practice,cert, confusing, cert,error-hand..."
9689,AWenw12KZ1vVHhGU4vEw,"[pitfall, convention]"
9690,AWenwUBsZ1vVHhGU4vAM,"[convention, brain-overload, bad-practice,cert]"
9691,AWenyFZ8Z1vVHhGU4vLr,"[bad-practice,cert,cwe,error-handling]"


This result shows the unique tag combinations associated with each analysis.

In [42]:
df_tags.loc[9688,"UNIQUE_TAGS"]

array(['bad-practice,cert', 'confusing', 'cert,error-handling,suspicious'],
      dtype=object)

As can be seen, the combinations lead to tags still appearing twice which needs to be further processed.

In [45]:
def safe_split(value):
    """Safely splits comma-separated strings within a NumPy array into unique tag values or returns an empty list."""
    if isinstance(value, np.ndarray):
        categories = []
        for item in value:
            if isinstance(item, str):
                categories.extend([cat.strip() for cat in item.split(',')])
        return list(set(categories))
    return []

df_tags["UNIQUE_TAGS"] = df_tags["UNIQUE_TAGS"].apply(safe_split)
df_tags

,CREATION_ANALYSIS_KEY,UNIQUE_TAGS
0,AV0-6-3jt6tne_r58pUF,"[bad-practice, suspicious, error-handling, cert]"
1,AV0-8Tovt6tne_r58pUY,[clumsy]
2,AV0-9p72t6tne_r58pUo,[bad-practice]
3,AV0-BQ2Et6tne_r58pCR,[design]
4,AV0-LoN2t6tne_r58pFZ,"[pitfall, cert]"
...,...,...
9688,AWentw92Z1vVHhGU4u0K,"[error-handling, bad-practice, confusing, cert..."
9689,AWenw12KZ1vVHhGU4vEw,"[pitfall, convention]"
9690,AWenwUBsZ1vVHhGU4vAM,"[bad-practice, brain-overload, convention, cert]"
9691,AWenyFZ8Z1vVHhGU4vLr,"[bad-practice, cwe, error-handling, cert]"


In [46]:
df_tags.loc[9688,"UNIQUE_TAGS"]

['error-handling', 'bad-practice', 'confusing', 'cert', 'suspicious']

Now the strings are properly split and the list doesn't contain duplicate values.

In [47]:
df_tags.to_csv(os.path.join(data_dir, 'tags_analysiskey_v2.csv'), index = False)